In [21]:
import os
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferWindowMemory, CombinedMemory, ConversationSummaryMemory, ChatMessageHistory
from langchain import hub
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent, Tool
import langdetect
from os import environ
from dotenv import load_dotenv

load_dotenv(dotenv_path='E:/Github_Repo/AI-Chatbot-Builder/backend/.env.dev', verbose=True)

True

### Requirements:
- Introduce local tourist attractions
- Set up travel plans from user requirements (duration, location, number of visitors, ...)
- Recommend places to visit, accommodations, food & beverage locations in surrounding areas

In [9]:
from googleapiclient.discovery import build

my_api_key = os.environ["GOOGLE_API_KEY"]  #The API_KEY you acquired
my_cse_id = os.environ["GOOGLE_CSE_ID"]  #The search-engine-ID you created


def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key) 
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']


results = google_search('tourist attractions in Hanoi', my_api_key, my_cse_id, num=3)
for result in results:
    print(result)

{'kind': 'customsearch#result', 'title': 'Must-Visit Attractions in Hanoi', 'htmlTitle': 'Must-<b>Visit Attractions in Hanoi</b>', 'link': 'https://www.traveloka.com/en-th/activities/find/must-visit-attractions-in-hanoi', 'displayLink': 'www.traveloka.com', 'snippet': 'Must-Visit Attractions in Hanoi · Thang Long Water Puppet Show Tickets: Skip The Line · Lotte World Aquarium Hanoi Ticket · Half Day Hanoi Foodie Evening Tour · The\xa0...', 'htmlSnippet': 'Must-<b>Visit Attractions in Hanoi</b> &middot; Thang Long Water Puppet Show Tickets: Skip The Line &middot; Lotte World Aquarium Hanoi Ticket &middot; Half Day Hanoi Foodie Evening Tour &middot; The&nbsp;...', 'cacheId': 'AWlUhDZ_KM0J', 'formattedUrl': 'https://www.traveloka.com/en-th/activities/.../must-visit-attractions-in-hanoi', 'htmlFormattedUrl': 'https://www.traveloka.com/en-th/activities/.../must-visit-<b>attractions-in-hanoi</b>', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSF

In [13]:
search = GoogleSearchAPIWrapper(k=5)
#weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=os.environ["OPENWEATHERMAP_API_KEY"])

tools = [
    Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    # Tool(
    #     name="weather",
    #     description="Get weather information.",
    #     func=weather.run)
        ]

In [14]:
search.results("tourist locations in Hanoi", 3)

[{'title': '7 Free Attractions to Visit in Hanoi For Budget Traveller',
  'link': 'https://www.traveloka.com/en-my/explore/destination/7-free-attractions-to-visit-in-hanoi-for-budget-traveller-trp/258733',
  'snippet': '6 thg 8, 2023 ... Best Budget-Friendly Attractions to Visit in Vietnam · 1. Hanoi Night Market. Hanoi offers several free attractions in Hanoi that you should not\xa0...'},
 {'title': 'Getting Around Hanoi, Vietnam',
  'link': 'https://www.traveloka.com/en-en/explore/destination/pl-cg-getting-around-hanoi-vietnam/230839',
  'snippet': '16 thg 5, 2023 ... Directly rent a car through Traveloka when you travel abroad to visit tourist attractions conveniently. Rent a Car with Traveloka. Rent a\xa0...'},
 {'title': '10 Best Things to Do in Hanoi',
  'link': 'https://www.traveloka.com/en-au/explore/activities/10-best-things-to-do-in-hanoi-trp/331570',
  'snippet': "8 thg 3, 2024 ... Want to experience the fun atmosphere in Hanoi? Visit Hoan Kiem Lake, a tourist destination in

In [17]:
weather.run("Huế, VN")

'In Huế, VN, the current weather is as follows:\nDetailed status: light intensity drizzle\nWind speed: 4.12 m/s, direction: 270°\nHumidity: 94%\nTemperature: \n  - Current: 21.06°C\n  - High: 21.06°C\n  - Low: 21.06°C\n  - Feels like: 21.68°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [17]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [18]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=1,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    max_tokens=2048
)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
agent_executor.invoke({"input": "Gợi ý cho tôi một vài món ăn, cũng như quán ăn ngon giá rẻ ở Huế"})



> Entering new AgentExecutor chain...
I should search for popular dishes and budget-friendly restaurants in Huế.
Action: google_search
Action Input: popular dishes in Hue Vietnam50A Hung Vuong, Phu Nhuan Ward, Hue City, Thua Thien Hue Province, Vietnam ... Savor your favorite dishes with special cuisines from Melia Vinpearl Hue ... Enjoy your foodie adventure by walking or riding a Vietnamese cyclo, a three-wheel bicycle taxi, driven by locals. Hue is popularly known for its royal cuisines ... ... Hue City, Thua Thien Hue Province, Vietnam ... Savor your favorite dishes with special cuisines from Carnosa Hotel Hue exclusively for you. ... Popular hotels in ... Savor your favorite dishes with special cuisines from Muong Thanh Holiday Hue Hotel exclusively for you. WiFi is available within public areas of the ... Savor your favorite dishes with special cuisines from My Home Homestay Hue exclusively for you. WiFi is available within public areas of the property to ...I should also searc

{'input': 'Gợi ý cho tôi một vài món ăn, cũng như quán ăn ngon giá rẻ ở Huế',
 'output': 'Some popular dishes in Huế are royal cuisines and bun bo Hue. Budget-friendly restaurants in Huế include Nice Hue Hotel and Cherish Hotel Hue.'}

In [20]:
agent_executor.invoke({"input": "Ngày mai liệu có phù hợp để đi chơi ở Huế không ?"})



> Entering new AgentExecutor chain...
I need to find out the weather forecast for Huế tomorrow to see if it's suitable for a day out.
Action: google_search
Action Input: "Huế weather forecast tomorrow"On the first night my bed and practically the whole room was dirty and humid, water heater switch was turned off, I showered with cold water in a 22°C weather. 28 thg 10, 2020 ... ... weather. Scope of application: Passengers departing and arriving at airports in Hue, Dong Hoi and Vinh with travel period from 20/10/2020 to ... Discover Ideas for Your Next Trip. Tours in Vietnam. Tours in Vietnam. Explore ... weather or traffic conditions. see more. General Information. Duration. 7 Hours ... What You'll Experience · Discover Ideas for Your Next Trip · Tour Itinerary · General Information · Frequently Asked Question · Other Activities. ... weather conditions ・Please ... Discover Ideas for Your Next Trip. Transports in Vietnam. Transports in Vietnam ... Scooter Rental in HueShuttle Bus fro

{'input': 'Ngày mai liệu có phù hợp để đi chơi ở Huế không ?',
 'output': 'The weather forecast for Huế tomorrow is not available through the search results, but I should check a reputable weather website directly for accurate information.'}

In [ ]:
# import requests
# 
# 
# def weather_search(location: str, search_type: str, date: str | None = None):
#     response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&appid={API key}")
#     print(response.json())